In [ ]:
from config import get_arguments
from SinGAN.manipulate import *
from SinGAN.training import *
import SinGAN.functions as functions

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

In [2]:
import easydict 
opt = easydict.EasyDict({
        'netG': '',
        'netD': '', 
        'manualSeed': 17,
        'nc_z':3,
        'nc_im':3,
        'out':'/home/s_246/Git/SinGAN/Outputs/',
        'nfc':32,
        'min_nfc':32,
        'ker_size':3,
        'num_layer':5,
        'stride':1,
        'padd_size':0,
        'scale_factor':0.75,
        'noise_amp':0.1,
        'min_size':25,
        'max_size':250,
        'niter': 2,
        'gamma':0.1,
        'lr_g':0.0005, #0.0005
        'lr_d':0.0005, #0.0005
        'beta1':0.5,
        'Gsteps':1,
        'Dsteps':1,
        'lambda_grad':0.1,
        'alpha':10,
        'input_dir':'/home/s_246/Git/SinGAN/SinGAN/Input/Images',
        'input_name':'balloons.png',
        'mode':'train'
})
opt

{'netG': '',
 'netD': '',
 'manualSeed': 17,
 'nc_z': 3,
 'nc_im': 3,
 'out': '/home/s_246/Git/SinGAN/Outputs/',
 'nfc': 32,
 'min_nfc': 32,
 'ker_size': 3,
 'num_layer': 5,
 'stride': 1,
 'padd_size': 0,
 'scale_factor': 0.75,
 'noise_amp': 0.1,
 'min_size': 25,
 'max_size': 250,
 'niter': 2,
 'gamma': 0.1,
 'lr_g': 0.0005,
 'lr_d': 0.0005,
 'beta1': 0.5,
 'Gsteps': 1,
 'Dsteps': 1,
 'lambda_grad': 0.1,
 'alpha': 10,
 'input_dir': '/home/s_246/Git/SinGAN/SinGAN/Input/Images',
 'input_name': 'balloons.png',
 'mode': 'train'}

In [3]:
from config import get_arguments
from SinGAN.manipulate import *
from SinGAN.training import *
import SinGAN.functions as functions

opt = functions.post_config(opt)
Gs = []
Zs = []
reals = []
NoiseAmp = []
dir2save = functions.generate_dir2save(opt)

if (os.path.exists(dir2save)):
    print('trained model already exist')
else:
    try:
        os.makedirs(dir2save)
    except OSError:
        pass
    
real = functions.read_image(opt)
functions.adjust_scales2image(real, opt) 

Random Seed:  17
trained model already exist


<tf.Tensor: shape=(1, 186, 248, 3), dtype=float32, numpy=
array([[[[-0.23137254,  0.13725495,  0.32549024],
         [-0.23137254,  0.13725495,  0.32549024],
         [-0.23137254,  0.13725495,  0.32549024],
         ...,
         [-0.30196077,  0.09019613,  0.3411765 ],
         [-0.30196077,  0.09019613,  0.3411765 ],
         [-0.3098039 ,  0.082353  ,  0.33333337]],

        [[-0.23137254,  0.13725495,  0.32549024],
         [-0.23137254,  0.13725495,  0.32549024],
         [-0.23137254,  0.13725495,  0.32549024],
         ...,
         [-0.30196077,  0.09019613,  0.3411765 ],
         [-0.32549018,  0.082353  ,  0.30980396],
         [-0.31764704,  0.07450986,  0.32549024]],

        [[-0.23137254,  0.13725495,  0.32549024],
         [-0.23137254,  0.13725495,  0.32549024],
         [-0.23137254,  0.13725495,  0.32549024],
         ...,
         [-0.27843136,  0.11372554,  0.36470592],
         [-0.27843136,  0.11372554,  0.36470592],
         [-0.29411763,  0.09803927,  0.3490196

## training 재현

In [4]:
from config import get_arguments
from SinGAN.manipulate import *
from SinGAN import training 
import SinGAN.functions as functions

import SinGAN.functions as functions
import SinGAN.models as models
import os
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import math
import matplotlib.pyplot as plt
from SinGAN.imresize import imresize
import tensorflow as tf
import pickle
import numpy as np


real_ = functions.read_image(opt)
in_s = 0
scale_num = 0
real = imresize(real_,opt.scale1,opt)
reals = functions.creat_reals_pyramid(real,reals,opt)
nfc_prev = 0
netD_optimizer = tf.keras.optimizers.Adam(learning_rate=opt.lr_d, beta_1=opt.beta1, beta_2=0.999) 
netG_optimizer = tf.keras.optimizers.Adam(learning_rate=opt.lr_g, beta_1=opt.beta1, beta_2=0.999) 

    
while scale_num<opt.stop_scale+1:
    opt.nfc = min(opt.nfc_init * pow(2, math.floor(scale_num / 4)), 128)
    opt.min_nfc = min(opt.min_nfc_init * pow(2, math.floor(scale_num / 4)), 128)
    opt.out_ = functions.generate_dir2save(opt)
    opt.outf = '%s/%d' % (opt.out_,scale_num)
    try:
        os.makedirs(opt.outf)
    except OSError:
            pass

    #plt.imsave('%s/in.png' %  (opt.out_), functions.convert_image_np(real), vmin=0, vmax=1)
    #plt.imsave('%s/original.png' %  (opt.out_), functions.convert_image_np(real_), vmin=0, vmax=1)
    plt.imsave('%s/real_scale.png' %  (opt.outf), functions.convert_image_np(reals[scale_num]), vmin=0, vmax=1)
    if (scale_num == 0) | (nfc_prev != opt.nfc):
        print('here1')
        D_curr,G_curr = init_models(opt) # 첫번째에서는 coarse model 이므로 모델을 초기화 한다.  #여기까지 ok!!!
    elif nfc_prev == opt.nfc: # 만약 첫번째 coarse model이 아니면..
        print('here2')
#         D_curr,G_curr = init_models(opt) # 첫번째에서는 coarse model 이므로 모델을 초기화 한다.  #여기까지 ok!!!
        D_curr.load_weights('%s/%d/netD' % (opt.out_, scale_num-1))
        G_curr.load_weights('%s/%d/netG' % (opt.out_, scale_num-1))
#         print('-------------')
#         print(D_curr.trainable_weights)
#         break
#         G_curr, D_curr = checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    #     G_curr.load_state_dict(torch.load('%s/%d/netG.pth' % (opt.out_,scale_num-1)))
    #     D_curr.load_state_dict(torch.load('%s/%d/netD.pth' % (opt.out_,scale_num-1)))
    else:
        print('error!')

    z_curr,in_s,G_curr,D_curr = training.train_single_scale(D_curr,G_curr,reals,Gs,Zs,in_s,NoiseAmp,opt,scale_num,netG_optimizer, netD_optimizer)
#     # G_curr = functions.reset_grads(G_curr,False)
#     # G_curr.eval()
#     # D_curr = functions.reset_grads(D_curr,False)
#     # D_curr.eval()
  
    Gs.append(G_curr)
    Zs.append(z_curr)
    NoiseAmp.append(opt.noise_amp)
    with open('%s/Zs.pkl' % (opt.out_), 'wb') as f:
        pickle.dump(Zs, f)
#     np.save('%s/Zs.npy' % (opt.out_), Zs, allow_pickle = False)
    # torch.save(Zs, '%s/Zs.pth' % (opt.out_)) 
    # torch.save(Gs, '%s/Gs.pth' % (opt.out_)) # functions.save_models에서 대신 save 수행 -> 추후 model load시 활용 
    with open('%s/reals.pkl' % (opt.out_), 'wb') as f:
        pickle.dump(reals, f)
#     np.save('%s/reals.npy' % (opt.out_), reals, allow_pickle = False)
    with open('%s/NoiseAmp.pkl' % (opt.out_), 'wb') as f:
        pickle.dump(NoiseAmp, f)
    # torch.save(reals, '%s/reals.pth' % (opt.out_))
#     np.save('%s/NoiseAmp' % (opt.out_), NoiseAmp, allow_pickle = False)
    # torch.save(NoiseAmp, '%s/NoiseAmp.pth' % (opt.out_))
    scale_num+=1
    nfc_prev = opt.nfc
#     break ################################################################################ 0 에서 끝남
#     del D_curr,G_curr

here1
errD_real: tf.Tensor(-0.020149307, shape=(), dtype=float32)
errD_fake: tf.Tensor(0.020163314, shape=(), dtype=float32)
errG_fake: tf.Tensor(-0.020163314, shape=(), dtype=float32)
rec_loss: tf.Tensor(1.5479393, shape=(), dtype=float32)
scale 0:[0/2]
errD_real: tf.Tensor(-0.02132981, shape=(), dtype=float32)
errD_fake: tf.Tensor(0.02113336, shape=(), dtype=float32)
errG_fake: tf.Tensor(-0.02113336, shape=(), dtype=float32)
rec_loss: tf.Tensor(1.3122678, shape=(), dtype=float32)
scale 0:[1/2]
here2
errD_real: tf.Tensor(-0.022512134, shape=(), dtype=float32)
errD_fake: tf.Tensor(0.022318989, shape=(), dtype=float32)
errG_fake: tf.Tensor(-0.022318989, shape=(), dtype=float32)
rec_loss: tf.Tensor(1.3670382, shape=(), dtype=float32)
scale 1:[0/2]
errD_real: tf.Tensor(-0.02430361, shape=(), dtype=float32)
errD_fake: tf.Tensor(0.024196217, shape=(), dtype=float32)
errG_fake: tf.Tensor(-0.024196217, shape=(), dtype=float32)
rec_loss: tf.Tensor(1.139769, shape=(), dtype=float32)
scale 1:[1/

In [5]:
SinGAN_generate(Gs,Zs,reals,NoiseAmp,opt)

<tf.Tensor: shape=(1, 186, 248, 3), dtype=float32, numpy=
array([[[[ 0.13238755,  0.1141886 , -0.10306969],
         [ 0.13888673,  0.11430264, -0.10298721],
         [ 0.14188355,  0.11434942, -0.10403344],
         ...,
         [ 0.14177658,  0.11501628, -0.10068817],
         [ 0.141213  ,  0.11473252, -0.10461721],
         [ 0.14098781,  0.11434985, -0.10332433]],

        [[ 0.13773236,  0.11461578, -0.10329923],
         [ 0.14039883,  0.1142568 , -0.10406666],
         [ 0.14079994,  0.11404531, -0.10474959],
         ...,
         [ 0.1351217 ,  0.11366731, -0.10148325],
         [ 0.13444944,  0.11413278, -0.10538308],
         [ 0.13523918,  0.11425439, -0.10399139]],

        [[ 0.14120044,  0.11421771, -0.10397033],
         [ 0.1408005 ,  0.11394194, -0.10422198],
         [ 0.14072962,  0.11477251, -0.10463542],
         ...,
         [ 0.13176632,  0.11390412, -0.10110962],
         [ 0.13176972,  0.11387261, -0.10548663],
         [ 0.13171327,  0.11434867, -0.1037564